In [35]:
#import libraries
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

SyntaxError: invalid syntax (3388273332.py, line 4)

In [17]:
words = []
classes = []
documents = []
ignore_words = ['?','!','.']

In [18]:
#call json file
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [30]:
#print(intents)

In [37]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/abhishek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/abhishek/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [29]:
# to identify the class of words
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print(pattern)
        #tokenize here
        w = nltk.word_tokenize(pattern)
        #print('token is: {}'.format(w))
        words.extend(w)
        documents.append((w, intent['tag']))
        # add tag to class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    #print('Word list is: {}'.format(words))
    #print('Docs are: {}'.format(documents))
    #print('Classes are: {}'.format(classes))

Classes are: ['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [38]:
lemmatizer = WordNetLemmatizer()

In [43]:
# lemmatisation
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = list(set(words)) # removes duplicates, first to set then to list
classes = list(set(classes))

In [44]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

Training and Creating model

In [45]:
training = []
output_empty = [0]*len(classes)

In [57]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #print('pattern words: {}'.format(pattern_words))
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    #print('Current bag: {}'.format(bag))
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print('current output: {}'.format(output_row))
    training.append([bag, output_row])
#print('training data: {}'.format(training))


In [60]:
import random
import numpy as np

In [63]:
# shuffle and convert to X and Y array
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])
#print('x: {}'.format(train_x))
#print('Y: {}'.format(train_y))

Create and Save Model

In [81]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [82]:
model = Sequential() # nodel created
# Adding Layers
model.add(Dense(128, input_shape = (len(train_x[0]),),activation='relu')) #hidden layer 1
model.add(Dropout(0.5)) # Dropout layer to avoid overfitting
model.add(Dense(64, activation = 'relu')) # anther layer
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax')) # output layer

In [83]:
# compiling and defining optimizer function
sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

mfit = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot_model.h5', mfit)
print('Model Done')

Epoch 1/200
282/282 [==============================] - 1s 1ms/step - loss: 1.0033 - accuracy: 0.6936
Epoch 2/200
282/282 [==============================] - 0s 1ms/step - loss: 0.1778 - accuracy: 0.9574
Epoch 3/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0735 - accuracy: 0.9844
Epoch 4/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0539 - accuracy: 0.9894
Epoch 5/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0205 - accuracy: 0.9957
Epoch 6/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0297 - accuracy: 0.9943
Epoch 7/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0144 - accuracy: 0.9986
Epoch 8/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0154 - accuracy: 0.9972
Epoch 9/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0162 - accuracy: 0.9965
Epoch 10/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0124 - accura

282/282 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 0.9993
Epoch 82/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 0.9993
Epoch 83/200
282/282 [==============================] - 0s 1ms/step - loss: 0.0081 - accuracy: 0.9972
Epoch 84/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 85/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 86/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0032 - accuracy: 0.9986
Epoch 87/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 88/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0036 - accuracy: 0.9979
Epoch 89/200
282/282 [==============================] - 1s 2ms/step - loss: 0.0023 - accuracy: 0.9993
Epoch 90/200
282/282 [==============================] - 0s 2ms/step - loss: 0.0045 - accuracy: 

Epoch 160/200
282/282 [==============================] - 0s 969us/step - loss: 3.8583e-04 - accuracy: 1.0000
Epoch 161/200
282/282 [==============================] - 0s 932us/step - loss: 5.7525e-04 - accuracy: 1.0000
Epoch 162/200
282/282 [==============================] - 0s 931us/step - loss: 0.0017 - accuracy: 0.9993
Epoch 163/200
282/282 [==============================] - 0s 975us/step - loss: 8.0146e-04 - accuracy: 1.0000
Epoch 164/200
282/282 [==============================] - 0s 965us/step - loss: 0.0019 - accuracy: 0.9986
Epoch 165/200
282/282 [==============================] - 0s 949us/step - loss: 5.5129e-04 - accuracy: 1.0000
Epoch 166/200
282/282 [==============================] - 0s 963us/step - loss: 0.0018 - accuracy: 0.9986
Epoch 167/200
282/282 [==============================] - 0s 938us/step - loss: 7.3104e-04 - accuracy: 1.0000
Epoch 168/200
282/282 [==============================] - 0s 934us/step - loss: 0.0013 - accuracy: 0.9993
Epoch 169/200
282/282 [============